In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/Cleaned4weekRealestateData.csv")

In [4]:
pip install statsmodels

   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ------------------------------ --------- 7.3/9.6 MB 50.6 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 43.0 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [patsy]
   ---------------------------------------- 0/2 [patsy]
   ---------------------------------------- 0/2 [patsy]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [statsmodels]
   -------------------- ------------------- 1/2 [s


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
df.head()

,PERIOD_BEGIN,PERIOD_END,REGION_TYPE,REGION_TYPE_ID,REGION_NAME,REGION_ID,ADJUSTED_AVERAGE_NEW_LISTINGS,ADJUSTED_AVERAGE_NEW_LISTINGS_YOY,AVERAGE_PENDING_SALES_LISTING_UPDATES,AVERAGE_PENDING_SALES_LISTING_UPDATES_YOY,...,AGE_OF_INVENTORY_YOY,WEEKS_OF_SUPPLY,WEEKS_OF_SUPPLY_YOY,MEDIAN_PENDING_SQFT,MEDIAN_PENDING_SQFT_YOY,AVERAGE_SALE_TO_LIST_RATIO,AVERAGE_SALE_TO_LIST_RATIO_YOY,MEDIAN_SALE_PPSF,MEDIAN_SALE_PPSF_YOY,LAST_UPDATED
0,2017-01-02,2017-01-29,metro,-2,"Aberdeen, SD metro area",10100,4.50,-0.419355,5.50,0.100000,...,-10.750,21.701389,-1.841468,2317.000,0.129624,0.991026,0.040947,82.612505,0.000237,2025-09-24 13:36:20.730
1,2017-01-09,2017-02-05,metro,-2,"Aberdeen, SD metro area",10100,3.25,-0.535714,6.25,0.250000,...,-3.875,17.034722,-7.174802,2400.625,0.160984,0.977997,0.035702,81.958745,0.072890,2025-09-24 13:36:20.730
2,2017-01-16,2017-02-12,metro,-2,"Aberdeen, SD metro area",10100,4.25,-0.370370,4.25,-0.055556,...,4.250,26.020833,-0.045833,2420.125,0.195271,0.978080,0.024140,80.368410,0.087961,2025-09-24 13:36:20.730
3,2017-01-23,2017-02-19,metro,-2,"Aberdeen, SD metro area",10100,5.50,-0.266667,3.75,-0.062500,...,7.500,32.708333,2.191667,2285.500,0.167784,0.964273,-0.007444,76.648096,-0.120064,2025-09-24 13:36:20.730
4,2017-01-30,2017-02-26,metro,-2,"Aberdeen, SD metro area",10100,7.25,0.115385,3.25,-0.187500,...,5.375,35.541667,4.575000,2459.000,0.354914,0.963298,0.022579,76.525164,-0.026367,2025-09-24 13:36:20.730


In [20]:
df["MEDIAN_SALE_PRICE",          # price level
    "MEDIAN_DAYS_ON_MARKET",      # liquidity speed
    "WEEKS_OF_SUPPLY",            # inventory tightness
    "AVERAGE_SALE_TO_LIST_RATIO", # pricing power
    "PERCENT_ACTIVE_LISTINGS_WITH_PRICE_DROPS",  # distress
    "ADJUSTED_AVERAGE_HOMES_SOLD",
     "ACTIVE_LISTINGS",
    "ADJUSTED_AVERAGE_HOMES_SOLD",
    "MEDIAN_NEW_LISTING_PRICE",
    "MEDIAN_SALE_PPSF",
    "AGE_OF_INVENTORY"]

KeyError: ('MEDIAN_SALE_PRICE', 'MEDIAN_DAYS_ON_MARKET', 'WEEKS_OF_SUPPLY', 'AVERAGE_SALE_TO_LIST_RATIO', 'PERCENT_ACTIVE_LISTINGS_WITH_PRICE_DROPS', 'ADJUSTED_AVERAGE_HOMES_SOLD', 'ACTIVE_LISTINGS', 'ADJUSTED_AVERAGE_HOMES_SOLD', 'MEDIAN_NEW_LISTING_PRICE', 'MEDIAN_SALE_PPSF', 'AGE_OF_INVENTORY')

In [ ]:

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_percentage_error

# Choose what you want to forecast forward
TARGETS = [
    "MEDIAN_SALE_PRICE",          # price level
    "MEDIAN_DAYS_ON_MARKET",      # liquidity speed
    "WEEKS_OF_SUPPLY",            # inventory tightness
    "AVERAGE_SALE_TO_LIST_RATIO", # pricing power
    "PERCENT_ACTIVE_LISTINGS_WITH_PRICE_DROPS",  # distress
    "ADJUSTED_AVERAGE_HOMES_SOLD"
]

# Exogenous regressors (contemporaneous + lagged will be created)
EXOG = [
    "ACTIVE_LISTINGS",
    "ADJUSTED_AVERAGE_HOMES_SOLD",
    "MEDIAN_NEW_LISTING_PRICE",
    "MEDIAN_SALE_PPSF",
    "AGE_OF_INVENTORY",
]

# Resample weekly → monthly (end-of-month) to reduce noise
def to_monthly(df):
    df = df.copy()
    df["PERIOD_BEGIN"] = pd.to_datetime(df["PERIOD_BEGIN"])
    df = df.set_index("PERIOD_BEGIN").sort_index()
    # For rates/ratios: mean; for levels: mean is fine here
    monthly = df.resample("M").mean()
    monthly.index.name = "DATE"
    return monthly

# Make supervised features (lags, moving avgs) for exogenous drivers
def make_exog(dfm, cols, lags=(1,2,3), mas=(3,6)):
    X = pd.DataFrame(index=dfm.index)
    for c in cols:
        if c not in dfm: 
            continue
        X[c] = dfm[c]
        for L in lags:
            X[f"{c}_lag{L}"] = dfm[c].shift(L)
        for k in mas:
            X[f"{c}_ma{k}"] = dfm[c].rolling(k, min_periods=1).mean()
    return X


In [15]:
def fit_ensemble(y, X, seasonal_periods=12):
    """
    Returns a dict with fitted SARIMAX & ETS. (Simple ensemble later.)
    y: monthly series (pd.Series)
    X: exogenous (aligned with y)
    """
    # Basic SARIMAX (auto-ARIMA is fine but we'll keep it deterministic)
    # (p,d,q)=(1,1,1) seasonal (1,1,1,12) is a robust monthly default; tune if needed.
    sarimax = SARIMAX(
        y, exog=X, order=(1,1,1), seasonal_order=(1,1,1,seasonal_periods),
        enforce_stationarity=False, enforce_invertibility=False
    ).fit(disp=False)

    # ETS (no exog) — strong for level/seasonality
    ets = ExponentialSmoothing(
        y, trend="add", seasonal="add", seasonal_periods=seasonal_periods, initialization_method="estimated"
    ).fit(optimized=True)

    return {"sarimax": sarimax, "ets": ets}

def forecast_ensemble(models, steps, X_future=None, alpha=0.2):
    """
    Weighted ensemble: 0.6 SARIMAX + 0.4 ETS (tweak if needed).
    Returns mean forecast & PI via simple combination of model variances.
    """
    sarimax = models["sarimax"]
    ets = models["ets"]

    # SARIMAX forecast (with future exog if provided)
    if X_future is not None and len(X_future) >= steps:
        fc_sar = sarimax.get_forecast(steps=steps, exog=X_future.iloc[:steps])
    else:
        fc_sar = sarimax.get_forecast(steps=steps)
    mean_sar = fc_sar.predicted_mean
    pi_sar = fc_sar.conf_int(alpha=alpha)

    # ETS forecast
    fc_ets = ets.forecast(steps)
    # crude PI: use in-sample residual std
    resid_std = np.std(ets.resid, ddof=1)
    z = 1.2816 if alpha==0.2 else 1.96   # ~80% default
    pi_ets = pd.DataFrame({
        "lower": fc_ets - z*resid_std,
        "upper": fc_ets + z*resid_std
    }, index=fc_ets.index)

    # combine
    w_sar, w_ets = 0.6, 0.4
    mean = w_sar*mean_sar + w_ets*fc_ets
    lower = w_sar*pi_sar.iloc[:,0] + w_ets*pi_ets["lower"]
    upper = w_sar*pi_sar.iloc[:,1] + w_ets*pi_ets["upper"]

    out = pd.DataFrame({"mean": mean, "lower": lower, "upper": upper})
    return out

def rolling_backtest(y, X, start_year=2021, seasonal_periods=12):
    """
    Expanding window backtest: fit up to t, forecast next 6 months, roll.
    Returns MAPE for each horizon and a dict of forecasts.
    """
    y = y.dropna()
    X = X.loc[y.index].copy() if X is not None else None

    results = []
    start_idx = y.index.get_loc(y.index[y.index.year>=start_year][0])

    for i in range(start_idx, len(y)-6):
        y_train = y.iloc[:i]
        X_train = X.iloc[:i] if X is not None else None

        mdl = fit_ensemble(y_train, X_train, seasonal_periods)
        X_future = X.iloc[i:i+6] if X is not None else None
        fc = forecast_ensemble(mdl, steps=6, X_future=X_future)

        actual = y.iloc[i:i+6]
        mape = mean_absolute_percentage_error(actual, fc["mean"].reindex(actual.index))
        results.append(mape)

    return np.mean(results) if results else np.nan


In [16]:
def metro_forecasts(df_metro, region_name, horizon_months=60, alpha=0.2):
    g = df_metro[df_metro["REGION_NAME"]==region_name].copy()
    g_m = to_monthly(g)

    # Build exog frame once
    X = make_exog(g_m, EXOG, lags=(1,2,3,6,12), mas=(3,6,12))
    # Prepare future exog by naïve extension (last known values) — or
    # use simple seasonal repeats:
    future_idx = pd.period_range(g_m.index[-1]+pd.offsets.MonthEnd(1), periods=horizon_months, freq="M").to_timestamp()
    X_future = pd.DataFrame(index=future_idx)
    for col in X.columns:
        # seasonal repeat by month-of-year average
        by_m = X.groupby(X.index.month)[col].mean()
        X_future[col] = [by_m.loc[d.month] for d in future_idx]

    out = {}
    scores = {}

    for target in TARGETS:
        if target not in g_m.columns: 
            continue
        y = g_m[target].astype(float)

        # Align exog
        X_aligned = X.loc[y.index]

        # Backtest quality
        cv_mape = rolling_backtest(y, X_aligned, start_year=2021, seasonal_periods=12)
        scores[target] = cv_mape

        # Fit on full history and forecast
        mdl = fit_ensemble(y, X_aligned, seasonal_periods=12)
        fc = forecast_ensemble(mdl, steps=horizon_months, X_future=X_future)
        out[target] = fc

    return out, scores


In [17]:
def score_from_forecasts(fc_dict):
    # All forecasts share the same future index; assemble into a frame
    idx = next(iter(fc_dict.values()))["mean"].index
    F = pd.DataFrame(index=idx)
    for k, dfk in fc_dict.items():
        F[k] = dfk["mean"]

    # Derived metrics
    F["Liquidity"] = 1 / F["MEDIAN_DAYS_ON_MARKET"]
    F["Tightness"] = 1 / F["WEEKS_OF_SUPPLY"]
    F["DemandStrength"] = F["AVERAGE_SALE_TO_LIST_RATIO"] / F["PERCENT_ACTIVE_LISTINGS_WITH_PRICE_DROPS"]
    F["Affordability"] = 1 / F["MEDIAN_SALE_PRICE"]

    # Normalize 0–1 per metric on the forecast horizon to combine
    def norm(s): 
        return (s - s.min()) / (s.max() - s.min() + 1e-9)

    for c in ["Liquidity","Tightness","DemandStrength","Affordability"]:
        F[c+"_n"] = norm(F[c])

    # Future iBuyer score (weights like your Stage-2)
    F["Future_iBuyer_Score"] = (
        0.35*F["Liquidity_n"] +
        0.25*F["DemandStrength_n"] +
        0.20*F["Tightness_n"] +
        0.20*F["Affordability_n"]
    )

    return F

def summarize_future_score(F):
    out = {}
    for horizon, months in [("2y", 24), ("5y", 60)]:
        span = F["Future_iBuyer_Score"].iloc[:months].dropna()
        if len(span)==0: 
            out[f"mean_{horizon}"] = np.nan
            out[f"p10_{horizon}"]  = np.nan
            out[f"std_{horizon}"]  = np.nan
        else:
            out[f"mean_{horizon}"] = span.mean()
            out[f"p10_{horizon}"]  = np.percentile(span, 10)
            out[f"std_{horizon}"]  = span.std(ddof=0)
    return out


In [18]:
SHORTLIST = [
    "Houston, TX metro area",
    "Atlanta, GA metro area",
    "Chicago, IL metro area",
    "Phoenix, AZ metro area",
    "Dallas, TX metro area",
    "Minneapolis, MN metro area",
]

all_rows = []
for metro in SHORTLIST:
    fc, cv = metro_forecasts(df, metro, horizon_months=60, alpha=0.2)   # 80% PI
    F = score_from_forecasts(fc)
    s = summarize_future_score(F)
    s.update({
        "REGION_NAME": metro,
        "MAPE_price": cv.get("MEDIAN_SALE_PRICE", np.nan),
        "MAPE_dom":   cv.get("MEDIAN_DAYS_ON_MARKET", np.nan),
    })
    all_rows.append(s)

future_rank = pd.DataFrame(all_rows).sort_values(["p10_2y","mean_2y"], ascending=[False, False])
print("\n🏁 Risk-adjusted ranking (2-year horizon):")
print(future_rank[["REGION_NAME","mean_2y","p10_2y","std_2y","MAPE_price","MAPE_dom"]].round(3).to_markdown(index=False))

print("\n🏁 Risk-adjusted ranking (5-year horizon):")
print(future_rank.sort_values(["p10_5y","mean_5y"], ascending=[False, False])
      [["REGION_NAME","mean_5y","p10_5y","std_5y"]].round(3).to_markdown(index=False))


C:\Users\megha\AppData\Local\Temp\ipykernel_8536\4040775332.py:29: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.resample("M").mean()


TypeError: agg function failed [how->mean,dtype->object]